In [1]:
import numpy as np
import pandas as pd
import json

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# This class is for the preprocessing, take csv path as input, and aim to return a pandas data frame for trainning
class PreProcessing:

  # The constructor takes a pandas dataframe as input and save it to self.df
  def __init__(self, csvpath):
    self.df = pd.read_csv(csvpath)
    if "train" in csvpath:
      self.dftype = 1
    elif "test" in csvpath:
      self.dftype = 2
    else:
      self.dftype = -1

  # This method have deal with missing data before merge or drop
  def MissingData(self):
    self.df = self.df.replace(-1, np.NaN) #first, replace -1 to NaN
    #print (self.df.columns[self.df.isnull().any()])
    '''
    'ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 
    'ps_reg_03',
    'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_05_cat', 'ps_car_07_cat', 'ps_car_09_cat', 
    'ps_car_11', 'ps_car_12', 'ps_car_14'
    '''
    mean_imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    mdan_imp = Imputer(missing_values='NaN', strategy='median', axis=0)
    mfrq_imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

    self.df["ps_ind_02_cat"].fillna(-1, inplace=True)
    self.df["ps_ind_04_cat"].fillna(-1, inplace=True)
    self.df["ps_ind_05_cat"].fillna(-1, inplace=True)
    #self.df["ps_reg_03"].fillna(self.df["ps_reg_03"].median(), inplace=True)
    #self.df['ps_reg_03'] = mean_imp.fit_transform(self.df[['ps_reg_03']]).ravel()
    self.df["ps_reg_03"].fillna(2 * self.df['ps_reg_03'].value_counts().idxmax(), inplace=True)
    self.df["ps_car_01_cat"].fillna(-1, inplace=True)
    self.df["ps_car_02_cat"].fillna(-1, inplace=True)
    #self.df["ps_car_03_cat"].fillna(self.df["ps_car_03_cat"].value_counts().idxmax(), inplace=True) # top 1 missing variable, drop
    #self.df["ps_car_05_cat"].fillna(self.df["ps_car_05_cat"].value_counts().idxmax(), inplace=True) # top 2 missing variable, drop
    self.df["ps_car_03_cat"].fillna(-1, inplace=True) # top 1 missing variable
    self.df["ps_car_05_cat"].fillna(-1, inplace=True) # top 2 missing variable
    self.df["ps_car_07_cat"].fillna(-1, inplace=True)
    self.df["ps_car_09_cat"].fillna(-1, inplace=True)
    #self.df["ps_car_11"].fillna(self.df["ps_car_11"].value_counts().idxmax(), inplace=True)
    self.df["ps_car_11"].fillna(-1, inplace=True) # this is catually a cat variable, inverted with ps_car_11_cat
    #self.df['ps_car_11'] = mfrq_imp.fit_transform(self.df[['ps_car_11']]).ravel()
    #self.df["ps_car_12"].fillna(self.df["ps_car_12"].median(), inplace=True)
    #self.df['ps_car_12'] = mean_imp.fit_transform(self.df[['ps_car_12']]).ravel()
    self.df['ps_car_12'].fillna(2 * self.df['ps_car_12'].value_counts().idxmax(), inplace=True)
    #self.df["ps_car_14"].fillna(self.df["ps_car_14"].median(), inplace=True)
    #self.df['ps_car_14'] = mean_imp.fit_transform(self.df[['ps_car_14']]).ravel()
    self.df["ps_car_14"].fillna(2 * self.df['ps_car_14'].value_counts().idxmax(), inplace=True)

    #self.df[""].fillna(self.df[""].mean(), inplace=True)
    #self.df[""].fillna(self.df[""].median(), inplace=True)
    #self.df[""].fillna(self.df[""].value_counts().idxmax(), inplace=True)
    return

  # This method drop the original catagory labels and replaced with one hot labels
  def OneHotReplacement(self):

    #onehot_cols = ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_07_cat', 'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11']
    onehot_cols = ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_07_cat', 'ps_car_09_cat', 'ps_car_11']

    self.df = pd.get_dummies(self.df, columns=onehot_cols, drop_first=True)
    #onehot = pd.get_dummies(self.df['ps_ind_02_cat'])
    #self.df.drop(['ps_ind_02_cat'], axis = 1, inplace = True)
    #self.df = self.df.join(onehot)
    return

  # This method drop or merge variables in dataframe accroding to corr map
  def CorrMergeDrop(self):
    #self.df['ps_ind_06070809_bin'] = self.df.apply(
    #  lambda x: 1 if x['ps_ind_06_bin'] == 1 
    #              else 
    #              (2 if x['ps_ind_07_bin'] == 1 
    #                 else 
    #                 ( 3 if x['ps_ind_08_bin'] == 1 
    #                     else 
    #                     (4 if x['ps_ind_09_bin'] == 1 else 5)
    #                 )
    #              ), axis = 1)
    #self.df.drop(['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin'], axis = 1, inplace = True)

    #self.df['ps_ind_161718_bin'] = self.df.apply(lambda x: 1 if x['ps_ind_16_bin'] == 1 
    #                                                         else (2 if x['ps_ind_17_bin'] == 1 else 3), axis = 1)
    #self.df.drop(['ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin'], axis = 1, inplace = True)
 
    # drop this variable from preprocessing study, top 3 missing data, and not important at all
    #self.df.drop(['ps_car_03_cat'], axis = 1, inplace = True)
    #self.df.drop(['ps_car_05_cat'], axis = 1, inplace = True)
    # drop less important features, random forest
    #'''
    #self.df.drop(['ps_calc_15_bin'], axis = 1, inplace = True)
    #self.df.drop(['ps_calc_16_bin'], axis = 1, inplace = True)
    #self.df.drop(['ps_calc_17_bin'], axis = 1, inplace = True)
    #self.df.drop(['ps_calc_18_bin'], axis = 1, inplace = True)
    #self.df.drop(['ps_calc_19_bin'], axis = 1, inplace = True)
    #self.df.drop(['ps_calc_20_bin'], axis = 1, inplace = True)
    self.df.drop(['ps_ind_10_bin'], axis = 1, inplace = True)
    self.df.drop(['ps_ind_11_bin'], axis = 1, inplace = True)
    self.df.drop(['ps_ind_12_bin'], axis = 1, inplace = True)
    self.df.drop(['ps_ind_13_bin'], axis = 1, inplace = True)
    #self.df.drop(['ps_ind_14'], axis = 1, inplace = True)
    #self.df.drop(['ps_car_04_cat'], axis = 1, inplace = True)
    self.df.drop(['ps_car_10_cat'], axis = 1, inplace = True)
    #'''
    #self.df['ps_car_13'] = (self.df['ps_car_13']*self.df['ps_car_13']*48400).round(0)
    #self.df['ps_car_12'] = (self.df['ps_car_12']*self.df['ps_car_12']).round(4) * 10000
    return

  # scale the features 
  def ScaleFeatures(self):
    scaler = StandardScaler(copy=False)
    if self.dftype == 1:
      scaler.fit_transform(self.df.drop(['id','target'], axis=1))
    elif self.dftype == 2:
      scaler.fit_transform(self.df.drop(['id'], axis=1))
    else:
      print ("neither train nor test!")
    return

  # this method pack all previous preprocessing all together and return the data frame
  def FinalFrameforTrainning(self):
    self.MissingData()
    self.OneHotReplacement()
    self.CorrMergeDrop()
    self.ScaleFeatures()
    #print (self.df)
    return self.df


In [2]:
preprocessing = PreProcessing('../data/train.csv')
train_p = preprocessing.FinalFrameforTrainning()
print ("done with trainning set preprocessing!")
#train_p.to_csv('train_p.csv', index = False)
preprocessing = PreProcessing('../data/test.csv')
test_p = preprocessing.FinalFrameforTrainning()
print ("done with test set preprocessing!")
#test_p.to_csv('test_p.csv', index = False)
#train_p = pd.read_csv('train_p.csv')
#test_p = pd.read_csv('test_p.csv')

done with trainning set preprocessing!
done with test set preprocessing!


In [3]:
#print(train_p.head())
print(train_p.shape)
print(test_p.shape)

(595212, 94)
(892816, 93)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import xgboost

'''
X_train, X_test, y_train, y_test = train_test_split(
                                                    train_p.drop(['id', 'target'],axis=1),
                                                    train_p.target,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=0
                                                   )
#xgb = XGBClassifier(    
#                    n_estimators=1000,
#                    max_depth=6,
#                    objective="binary:logistic",
#                    learning_rate=0.07, 
#                    subsample=.8,
#                    min_child_weight=.77,
#                    colsample_bytree=.8,
#                    scale_pos_weight=1.6,
#                    gamma=10,
#                    reg_alpha=8,
#                    reg_lambda=1.3,
#                    )

xgb = XGBClassifier(n_estimators=1000, objective="binary:logistic", learning_rate=0.014, max_depth=4, min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,  reg_alpha=0, reg_lambda=1, nthread=6)
                    
xgb.fit(X_train, y_train)
print("----- Training Done -----")
ysc_pred = xgb.predict_proba(X_test)[:,1]
#print (ysc_pred)
#print (y_test)
gini = 2*roc_auc_score(y_test, ysc_pred)-1
print(gini)
'''

/home/hua/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'\nX_train, X_test, y_train, y_test = train_test_split(\n                                                    train_p.drop([\'id\', \'target\'],axis=1),\n                                                    train_p.target,\n                                                    train_size=0.7,\n                                                    test_size=0.3,\n                                                    random_state=0\n                                                   )\n#xgb = XGBClassifier(    \n#                    n_estimators=1000,\n#                    max_depth=6,\n#                    objective="binary:logistic",\n#                    learning_rate=0.07, \n#                    subsample=.8,\n#                    min_child_weight=.77,\n#                    colsample_bytree=.8,\n#                    scale_pos_weight=1.6,\n#                    gamma=10,\n#                    reg_alpha=8,\n#                    reg_lambda=1.3,\n#                    )\n\nxgb = XGBClassifier(n_es

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [6]:
features = train_p.drop(['id', 'target'],axis=1).columns.values
x, y = (list(x) for x in zip(*sorted(zip(xgb.feature_importances_, features), 
                                                            reverse = False)))
#print(y)
indice = 0
for thisy in y:
    if 'ps_ind' in thisy:
        print(indice)
        print(y[indice])
        print(x[indice])
    indice = indice+1

trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='XGBoost Feature importance',
    orientation='h',
)

layout = dict(
    title='Feature importances, XGBoost',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
#         domain=[0, 0.85],
    ))

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

NameError: name 'xgb' is not defined

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
def xgb_param_selection(nfolds):
    #sc_mod = XGBClassifier(n_estimators=100, objective="binary:logistic", learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, nthread=6)
    #sc_mod = XGBClassifier(n_estimators=100, objective="binary:logistic", learning_rate=0.1, max_depth=4, min_child_weight=5, subsample=0.8, colsample_bytree=0.8, nthread=6)
    #sc_mod = XGBClassifier(n_estimators=100, objective="binary:logistic", learning_rate=0.1, max_depth=4, min_child_weight=5, gamma=0, nthread=6)
    #sc_mod = XGBClassifier(n_estimators=100, objective="binary:logistic", learning_rate=0.1, max_depth=4, min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6)
    #sc_mod = XGBClassifier(objective="binary:logistic", max_depth=4, min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8, reg_alpha=0, reg_lambda=1, nthread=6)
    #sc_mod = XGBClassifier(n_estimators=1000, objective="binary:logistic", learning_rate=0.14, subsample=0.8, colsample_bytree=0.8, nthread=6)
    #sc_mod = XGBClassifier(n_estimators=400, objective="binary:logistic", learning_rate=0.07, max_depth=4, min_child_weight=6, gamma=10, subsample=0.8, colsample_bytree=0.8, reg_alpha=8, reg_lambda=1.3, nthread=6)
    #sc_mod = XGBClassifier(n_estimators=700, objective="binary:logistic", learning_rate=0.025, max_depth=7, min_child_weight=10, gamma=0.65, subsample=0.8, colsample_bytree=0.4, max_delta_step=1.8, nthread=6)
    sc_mod = XGBClassifier(n_estimators=400, objective="binary:logistic", learning_rate=0.07, max_depth=5, min_child_weight=100, gamma=1.5, subsample=0.9, colsample_bytree=0.9, nthread=6)
    
    sposw = [1.0]
    param_grid = {'scale_pos_weight': sposw}    
    #mchdw = [1,3,5]
    #mdeps = [3,4,5,6,8,10,12]
    #mchdw = [1,2,3,4,5,6,7]
    #mdeps = [4]
    #mchdw = [5]
    #mdeps = [4]
    #param_grid = {'min_child_weight' : mchdw, 'max_depth': mdeps}
    #mnsps = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25]
    #param_grid = {'gamma' : mnsps}
    #subss = [0.6, 0.7, 0.8, 0.9]
    #colst = [0.6, 0.7, 0.8, 0.9]
    #param_grid = {'subsample' : subss, 'colsample_bytree': colst}
    #regl1 = [0]
    #regl2 = [0, 0.01, 0.1, 1, 10, 100]
    #regl1 = [0, 0.1, 1, 10]
    #regl2 = [0.1, 1]
    #param_grid = {'reg_alpha' : regl1, 'reg_lambda': regl2}
    #nests = [100, 200, 500, 1000]
    #lrate = [0.01, 0.1, 1]
    #nests = [1000]
    #lrate = [0.005, 0.01, 0.015, 0.02]
    #nests = [1000]
    #lrate = [0.011, 0.012, 0.013, 0.014, 0.015]      
    #param_grid = {'n_estimators': nests, 'learning_rate' : lrate}
    grid_search = GridSearchCV(sc_mod, param_grid, scoring='roc_auc', cv=nfolds, verbose=2)
    grid_search.fit(train_p.drop(['id', 'target'],axis=1), train_p.target)
    #print ("CV results")
    #print (grid_search.cv_results_)
    print ("Grid Scores:")
    print (grid_search.grid_scores_)
    print ("Best parameters:")
    print (grid_search.best_params_)
    return grid_search.cv_results_

In [8]:
xgb_param_selection(3)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] scale_pos_weight=1.0 ............................................
[CV] ............................. scale_pos_weight=1.0, total= 1.5min
[CV] scale_pos_weight=1.0 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s


[CV] ............................. scale_pos_weight=1.0, total= 1.5min
[CV] scale_pos_weight=1.0 ............................................
[CV] ............................. scale_pos_weight=1.0, total= 1.5min


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.6min finished


Grid Scores:
[mean: 0.63849, std: 0.00271, params: {'scale_pos_weight': 1.0}]
Best parameters:
{'scale_pos_weight': 1.0}


/home/hua/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning:

The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20

/home/hua/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning:

You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True

/home/hua/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning:

You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True

/home/hua/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning:

You are accessing a training score ('split1_train_score'), which 

{'mean_fit_time': array([ 89.01271876]),
 'mean_score_time': array([ 1.01508403]),
 'mean_test_score': array([ 0.63849366]),
 'mean_train_score': array([ 0.71148345]),
 'param_scale_pos_weight': masked_array(data = [1.0],
              mask = [False],
        fill_value = ?),
 'params': [{'scale_pos_weight': 1.0}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([ 0.63586876]),
 'split0_train_score': array([ 0.71242927]),
 'split1_test_score': array([ 0.6422216]),
 'split1_train_score': array([ 0.70986799]),
 'split2_test_score': array([ 0.63739063]),
 'split2_train_score': array([ 0.7121531]),
 'std_fit_time': array([ 1.45276944]),
 'std_score_time': array([ 0.00849706]),
 'std_test_score': array([ 0.00270828]),
 'std_train_score': array([ 0.00114786])}

In [10]:
# final block, rm after tuning
xgb_final = XGBClassifier(n_estimators=1000, objective="binary:logistic", learning_rate=0.014, max_depth=4, min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,  reg_alpha=0, reg_lambda=1, nthread=6)
xgb_final.fit(train_p.drop(['id', 'target'],axis=1), train_p.target)
print("----- Training Done -----")
pred_final = xgb_final.predict_proba(test_p.drop(['id'],axis=1))
print(pred_final)

----- Training Done -----
[[ 0.97268504  0.02731498]
 [ 0.97487372  0.02512625]
 [ 0.9721278   0.0278722 ]
 ..., 
 [ 0.96019721  0.03980282]
 [ 0.97476858  0.02523141]
 [ 0.97046328  0.02953671]]


In [12]:
#generate submission file
sub = pd.DataFrame()
sub['id'] = test_p.id
sub['target'] = pred_final[:,1]
sub.to_csv('../data/res/submit_PreOneHotDrop_XGB_20171127.csv', index=False)
sub.head()

,id,target
0,0,0.027315
1,1,0.025126
2,2,0.027872
3,3,0.015160
4,4,0.034867
